# Eat Safe, Love

## Notebook Set Up

In [1]:
from pymongo import MongoClient
import pandas as pd
from pprint import pprint

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [4]:
# review the collections in our database
collection_names = db.list_collection_names()
collection_names

['establishments']

In [5]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [8]:
# Find the establishments with a hygiene score of 20
query = {'scores.Hygiene': 20}
results = establishments.find(query)
# Use count documents to display the number of documents in the result
num_establishments = establishments.count_documents(query)
print(f"Number of establishments with a hygiene score of 20: {num_establishments}")
# Disply the first document in the results using pprint
pprint(results[0])

Number of establishments with a hygiene score of 20: 0


IndexError: no such item for Cursor instance

In [ ]:
# Convert the result to a Pandas DataFrame
df = pd.DataFrame(list(results))
# Display the number of rows in the DataFrame
num_rows = df.shape[0]
print(f"Number of rows in the DataFrame: {num_rows}")
# Display the first 10 rows of the DataFrame
df.head(10)

### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [ ]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
rv = {"RatingValue": {"$gte": 4}, "LocalAuthorityName": {"$regex": "London"}}
result = establishments.find(rv)
# Use count_documents to display the number of documents in the result
docs = establishments.count_documents(rv)
print(f"Number of establishments with a RatingValue greater than or equal to 4: {docs}")
# Display the first document in the results using pprint
pprint(result[0])

In [ ]:
# Convert the result to a Pandas DataFrame
df1 = pd.DataFrame(result)
# Display the number of rows in the DataFrame
rows = df1.shape[0]
print(f"Number of rows in the DataFrame: {rows}")
# Display the first 10 rows of the DataFrame
df1.head(10)

### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [ ]:
info = establishments.find_one({"BusinessName": "Penang Flavours"})
pprint(info)

In [ ]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score

degree_search = 0.01
latitude = 51.490142
longitude = 0.08384

query ={"RatingValue": 5,
        "geocode.latitude": {"$gte": (latitude - degree_search), "$lt": (latitude + degree_search)},
        "geocode.longitude": {"$gte": (longitude - degree_search), "$lt": (longitude + degree_search)}
       }

sort = [("scores.Hygiene", 1)]
limit = 5
# Print the results
res =list(establishments.find(query).sort(sort).limit(limit))
pprint(res)
    

In [ ]:
# Convert result to Pandas DataFrame
df2 = pd.DataFrame(res)
df2

### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [ ]:
# Create a pipeline that:
# 1. Matches establishments with a hygiene score of 0
# 2. Groups the matches by Local Authority
# 3. Sorts the matches from highest to lowest

match_query = {'$match':
               {"scores.Hygiene":0}
              }

group_query = {'$group':
               {'_id': {"classification": "$LocalAuthorityName", "culture": "$culture"},
                'count': {'$sum': 1 }
               }
              }

sort_values = {'$sort': {'count': -1}}


pipeline = [match_query, group_query, sort_values]
# Perform the aggregation
res1 = establishments.aggregate(pipeline)

# Print the number of documents in the result
res1 = list(res1)
print(len(res1))



# Print the first 10 results
pprint(res1[0:10])






In [ ]:
# Convert the result to a Pandas DataFrame
df3 = pd.DataFrame(res1)
# Display the number of rows in the DataFrame
rows2 = df3.shape[0]
print(f"Number of rows in the DataFrame: {rows2}")
# Display the first 10 rows of the DataFrame
df3.head(10)